In [ ]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import torchvision.transforms as T
from PIL import Image, ImageCms
import cv2
import skimage
import matplotlib.pyplot as plt

from scipy.stats import multivariate_normal

import sys

import scipy

from tqdm import tqdm

In [ ]:
np.set_printoptions(suppress=True)
eps = np.finfo(float).eps

In [ ]:
sys.path.append("rgb-lab")

In [ ]:
from rgb2lab import read_image, rgb2lab

In [ ]:
image_path = "dataset"

In [ ]:
NUM_IMAGES = 10000

In [ ]:
size_thresh = 0.01

In [ ]:
bins_str = """10   -20     0
10   -20    20
10     0   -20
10     0     0
10     0    20
10    20   -60
10    20   -40
10    20   -20
10    20     0
10    20    20
10    40   -60
10    40   -40
10    40   -20
10    40     0
10    40    20
10    60   -80
10    60   -60
30   -40     0
30   -40    20
30   -40    40
30   -20   -20
30   -20     0
30   -20    20
30   -20    40
30     0   -40
30     0   -20
30     0     0
30     0    20
30     0    40
30    20   -60
30    20   -40
30    20   -20
30    20     0
30    20    20
30    20    40
30    40   -80
30    40   -60
30    40   -40
30    40   -20
30    40     0
30    40    20
30    40    40
30    40    60
30    60  -100
30    60   -80
30    60   -60
30    60   -40
30    60   -20
30    60     0
30    60    20
30    60    40
30    60    60
30    80  -100
30    80   -80
30    80   -60
30    80   -40
50   -60    20
50   -60    40
50   -60    60
50   -40   -20
50   -40     0
50   -40    20
50   -40    40
50   -40    60
50   -20   -40
50   -20   -20
50   -20     0
50   -20    20
50   -20    40
50   -20    60
50     0   -60
50     0   -40
50     0   -20
50     0     0
50     0    20
50     0    40
50     0    60
50    20   -80
50    20   -60
50    20   -40
50    20   -20
50    20     0
50    20    20
50    20    40
50    20    60
50    40  -100
50    40   -80
50    40   -60
50    40   -40
50    40   -20
50    40     0
50    40    20
50    40    40
50    40    60
50    60  -100
50    60   -80
50    60   -60
50    60   -40
50    60   -20
50    60     0
50    60    20
50    60    40
50    60    60
50    80  -100
50    80   -80
50    80   -60
50    80   -40
50    80   -20
50    80     0
50    80    20
50    80    40
50    80    60
50   100   -80
50   100   -60
50   100   -40
50   100   -20
70   -80    40
70   -80    60
70   -80    80
70   -60     0
70   -60    20
70   -60    40
70   -60    60
70   -60    80
70   -40   -40
70   -40   -20
70   -40     0
70   -40    20
70   -40    40
70   -40    60
70   -40    80
70   -20   -40
70   -20   -20
70   -20     0
70   -20    20
70   -20    40
70   -20    60
70   -20    80
70     0   -60
70     0   -40
70     0   -20
70     0     0
70     0    20
70     0    40
70     0    60
70     0    80
70    20   -60
70    20   -40
70    20   -20
70    20     0
70    20    20
70    20    40
70    20    60
70    20    80
70    40   -60
70    40   -40
70    40   -20
70    40     0
70    40    20
70    40    40
70    40    60
70    40    80
70    60   -60
70    60   -40
70    60   -20
70    60     0
70    60    20
70    60    40
70    60    60
70    60    80
70    80   -60
70    80   -40
70    80   -20
70    80     0
70   100   -60
70   100   -40
90   -80    20
90   -80    40
90   -80    60
90   -80    80
90   -60   -20
90   -60     0
90   -60    20
90   -60    40
90   -60    60
90   -60    80
90   -40   -20
90   -40     0
90   -40    20
90   -40    40
90   -40    60
90   -40    80
90   -40   100
90   -20   -40
90   -20   -20
90   -20     0
90   -20    20
90   -20    40
90   -20    60
90   -20    80
90   -20   100
90     0   -40
90     0   -20
90     0     0
90     0    20
90     0    40
90     0    60
90     0    80
90     0   100
90    20   -40
90    20   -20
90    20     0
90    20    20
90    20    40
90    20    60
90    20    80
90    40   -20
90    40     0"""

In [ ]:
def matrix_str_to_np(str, type=int):
  matrix = np.array([list(map(type, filter(lambda x: x!= '', line.split(" ")))) for line in str.split("\n")])
  return matrix

In [ ]:
bins = matrix_str_to_np(bins_str)
bins.shape

In [ ]:
"""
d[i, j] = np.sqrt(np.sum(bins[i] - bins[j])^2)
so d[i, j] is the euclidean distance between bins[i] and bins[j]
"""
bin_count = bins.shape[0]
d = np.zeros((bin_count, bin_count))
for i in range(0, bin_count):
  for j in range(0, bin_count):
    d[i, j] = np.sqrt(np.sum((bins[i, :] - bins[j, :])**2))

In [ ]:
d_vec = d.flatten()

In [ ]:
min_dist = np.min(d_vec[d_vec > 0])
min_dist

In [ ]:
centers_str = """0     0   -10
0     0     0
0    10   -30
0    10   -20
0    10   -10
0    10     0
0    20   -30
0    20   -20
0    20   -10
0    20    10
0    30   -40
0    30   -30
10   -20     0
10   -20    20
10   -10   -20
10   -10   -10
10   -10     0
10   -10    10
10   -10    20
10     0   -20
10     0   -10
10     0     0
10     0    10
10     0    20
10    10   -30
10    10   -20
10    10   -10
10    10     0
10    10    20
10    20   -40
10    20   -30
10    20   -20
10    20   -10
10    20     0
10    20    10
10    30   -50
10    30   -40
10    30   -30
10    30   -20
10    30   -10
10    30     0
10    30    10
10    30    20
10    40   -60
10    40   -50
10    40   -40
10    40   -30
10    40   -20
10    40   -10
10    40     0
10    40    10
10    50   -70
10    50   -60
20   -30    10
20   -30    20
20   -30    30
20   -20     0
20   -20    10
20   -20    20
20   -20    30
20   -10   -20
20   -10   -10
20   -10     0
20   -10    10
20   -10    20
20   -10    30
20     0   -30
20     0   -20
20     0   -10
20     0     0
20     0    10
20     0    20
20     0    30
20    10   -40
20    10   -30
20    10   -20
20    10   -10
20    10     0
20    10    10
20    10    20
20    10    30
20    20   -50
20    20   -40
20    20   -30
20    20   -20
20    20   -10
20    20     0
20    20    10
20    20    20
20    20    30
20    30   -60
20    30   -50
20    30   -40
20    30   -30
20    30   -20
20    30   -10
20    30     0
20    30    10
20    30    20
20    30    30
20    40   -70
20    40   -60
20    40   -50
20    40   -40
20    40   -30
20    40   -20
20    40   -10
20    40     0
20    40    10
20    40    20
20    40    30
20    40    40
20    50   -80
20    50   -70
20    50   -60
20    50   -50
20    50   -40
20    50   -30
20    50   -20
20    50   -10
20    50     0
20    50    20
20    50    40
20    60   -90
20    60   -80
20    60   -70
20    60   -60
20    60   -50
20    70   -90
30   -40    20
30   -40    30
30   -40    40
30   -30     0
30   -30    10
30   -30    20
30   -30    30
30   -30    40
30   -20   -10
30   -20     0
30   -20    10
30   -20    20
30   -20    30
30   -20    40
30   -10   -20
30   -10   -10
30   -10     0
30   -10    10
30   -10    20
30   -10    30
30   -10    40
30     0   -40
30     0   -30
30     0   -20
30     0   -10
30     0     0
30     0    10
30     0    20
30     0    30
30     0    40
30    10   -50
30    10   -40
30    10   -30
30    10   -20
30    10   -10
30    10     0
30    10    10
30    10    20
30    10    30
30    10    40
30    20   -60
30    20   -50
30    20   -40
30    20   -30
30    20   -20
30    20   -10
30    20     0
30    20    10
30    20    20
30    20    30
30    20    40
30    30   -70
30    30   -60
30    30   -50
30    30   -40
30    30   -30
30    30   -20
30    30   -10
30    30     0
30    30    10
30    30    20
30    30    30
30    30    40
30    30    50
30    40   -80
30    40   -70
30    40   -60
30    40   -50
30    40   -40
30    40   -30
30    40   -20
30    40   -10
30    40     0
30    40    10
30    40    20
30    40    30
30    40    40
30    40    50
30    50   -90
30    50   -80
30    50   -70
30    50   -60
30    50   -50
30    50   -40
30    50   -30
30    50   -20
30    50   -10
30    50     0
30    50    10
30    50    20
30    50    30
30    50    40
30    50    50
30    60  -100
30    60   -90
30    60   -80
30    60   -70
30    60   -60
30    60   -50
30    60   -40
30    60   -30
30    60   -20
30    60   -10
30    60     0
30    60    10
30    60    20
30    60    30
30    60    40
30    60    50
30    70  -110
30    70  -100
30    70   -90
30    70   -80
30    70   -70
30    70   -60
30    70   -50
30    70   -40
30    80  -110
30    80  -100
30    80   -90
40   -50    30
40   -50    40
40   -50    50
40   -40    10
40   -40    20
40   -40    30
40   -40    40
40   -40    50
40   -30   -10
40   -30     0
40   -30    10
40   -30    20
40   -30    30
40   -30    40
40   -30    50
40   -20   -20
40   -20   -10
40   -20     0
40   -20    10
40   -20    20
40   -20    30
40   -20    40
40   -20    50
40   -10   -30
40   -10   -20
40   -10   -10
40   -10     0
40   -10    10
40   -10    20
40   -10    30
40   -10    40
40   -10    50
40     0   -50
40     0   -40
40     0   -30
40     0   -20
40     0   -10
40     0     0
40     0    10
40     0    20
40     0    30
40     0    40
40     0    50
40    10   -60
40    10   -50
40    10   -40
40    10   -30
40    10   -20
40    10   -10
40    10     0
40    10    10
40    10    20
40    10    30
40    10    40
40    10    50
40    20   -70
40    20   -60
40    20   -50
40    20   -40
40    20   -30
40    20   -20
40    20   -10
40    20     0
40    20    10
40    20    20
40    20    30
40    20    40
40    20    50
40    30   -80
40    30   -70
40    30   -60
40    30   -50
40    30   -40
40    30   -30
40    30   -20
40    30   -10
40    30     0
40    30    10
40    30    20
40    30    30
40    30    40
40    30    50
40    40   -90
40    40   -80
40    40   -70
40    40   -60
40    40   -50
40    40   -40
40    40   -30
40    40   -20
40    40   -10
40    40     0
40    40    10
40    40    20
40    40    30
40    40    40
40    40    50
40    40    60
40    50   -90
40    50   -80
40    50   -70
40    50   -60
40    50   -50
40    50   -40
40    50   -30
40    50   -20
40    50   -10
40    50     0
40    50    10
40    50    20
40    50    30
40    50    40
40    50    50
40    50    60
40    60  -100
40    60   -90
40    60   -80
40    60   -70
40    60   -60
40    60   -50
40    60   -40
40    60   -30
40    60   -20
40    60   -10
40    60     0
40    60    10
40    60    20
40    60    30
40    60    40
40    60    50
40    60    60
40    70  -100
40    70   -90
40    70   -80
40    70   -70
40    70   -60
40    70   -50
40    70   -40
40    70   -30
40    70   -20
40    70   -10
40    70     0
40    70    10
40    70    20
40    70    30
40    70    40
40    70    50
40    70    60
40    80  -100
40    80   -90
40    80   -80
40    80   -70
40    80   -60
40    80   -50
40    80   -40
40    80   -30
40    90   -90
50   -60    50
50   -60    60
50   -50    20
50   -50    30
50   -50    40
50   -50    50
50   -50    60
50   -40     0
50   -40    10
50   -40    20
50   -40    30
50   -40    40
50   -40    50
50   -40    60
50   -30   -20
50   -30   -10
50   -30     0
50   -30    10
50   -30    20
50   -30    30
50   -30    40
50   -30    50
50   -30    60
50   -20   -30
50   -20   -20
50   -20   -10
50   -20     0
50   -20    10
50   -20    20
50   -20    30
50   -20    40
50   -20    50
50   -20    60
50   -10   -40
50   -10   -30
50   -10   -20
50   -10   -10
50   -10     0
50   -10    10
50   -10    20
50   -10    30
50   -10    40
50   -10    50
50   -10    60
50     0   -50
50     0   -40
50     0   -30
50     0   -20
50     0   -10
50     0     0
50     0    10
50     0    20
50     0    30
50     0    40
50     0    50
50     0    60
50    10   -60
50    10   -50
50    10   -40
50    10   -30
50    10   -20
50    10   -10
50    10     0
50    10    10
50    10    20
50    10    30
50    10    40
50    10    50
50    10    60
50    20   -70
50    20   -60
50    20   -50
50    20   -40
50    20   -30
50    20   -20
50    20   -10
50    20     0
50    20    10
50    20    20
50    20    30
50    20    40
50    20    50
50    20    60
50    30   -80
50    30   -70
50    30   -60
50    30   -50
50    30   -40
50    30   -30
50    30   -20
50    30   -10
50    30     0
50    30    10
50    30    20
50    30    30
50    30    40
50    30    50
50    30    60
50    40   -90
50    40   -80
50    40   -70
50    40   -60
50    40   -50
50    40   -40
50    40   -30
50    40   -20
50    40   -10
50    40     0
50    40    10
50    40    20
50    40    30
50    40    40
50    40    50
50    40    60
50    50   -90
50    50   -80
50    50   -70
50    50   -60
50    50   -50
50    50   -40
50    50   -30
50    50   -20
50    50   -10
50    50     0
50    50    10
50    50    20
50    50    30
50    50    40
50    50    50
50    50    60
50    60   -90
50    60   -80
50    60   -70
50    60   -60
50    60   -50
50    60   -40
50    60   -30
50    60   -20
50    60   -10
50    60     0
50    60    10
50    60    20
50    60    30
50    60    40
50    60    50
50    60    60
50    60    70
50    70   -90
50    70   -80
50    70   -70
50    70   -60
50    70   -50
50    70   -40
50    70   -30
50    70   -20
50    70   -10
50    70     0
50    70    10
50    70    20
50    70    30
50    70    40
50    70    50
50    70    60
50    70    70
50    80   -90
50    80   -80
50    80   -70
50    80   -60
50    80   -50
50    80   -40
50    80   -30
50    80   -20
50    80   -10
50    80     0
50    80    10
50    80    20
50    80    30
50    80    40
50    80    50
50    80    60
50    80    70
50    90   -90
50    90   -80
50    90   -70
50    90   -60
50    90   -50
50    90   -40
50    90   -30
50    90   -20
50    90   -10
60   -70    60
60   -60    30
60   -60    40
60   -60    50
60   -60    60
60   -50    10
60   -50    20
60   -50    30
60   -50    40
60   -50    50
60   -50    60
60   -50    70
60   -40   -10
60   -40     0
60   -40    10
60   -40    20
60   -40    30
60   -40    40
60   -40    50
60   -40    60
60   -40    70
60   -30   -20
60   -30   -10
60   -30     0
60   -30    10
60   -30    20
60   -30    30
60   -30    40
60   -30    50
60   -30    60
60   -30    70
60   -20   -40
60   -20   -30
60   -20   -20
60   -20   -10
60   -20     0
60   -20    10
60   -20    20
60   -20    30
60   -20    40
60   -20    50
60   -20    60
60   -20    70
60   -10   -50
60   -10   -40
60   -10   -30
60   -10   -20
60   -10   -10
60   -10     0
60   -10    10
60   -10    20
60   -10    30
60   -10    40
60   -10    50
60   -10    60
60   -10    70
60     0   -60
60     0   -50
60     0   -40
60     0   -30
60     0   -20
60     0   -10
60     0     0
60     0    10
60     0    20
60     0    30
60     0    40
60     0    50
60     0    60
60     0    70
60    10   -70
60    10   -60
60    10   -50
60    10   -40
60    10   -30
60    10   -20
60    10   -10
60    10     0
60    10    10
60    10    20
60    10    30
60    10    40
60    10    50
60    10    60
60    10    70
60    20   -70
60    20   -60
60    20   -50
60    20   -40
60    20   -30
60    20   -20
60    20   -10
60    20     0
60    20    10
60    20    20
60    20    30
60    20    40
60    20    50
60    20    60
60    20    70
60    30   -70
60    30   -60
60    30   -50
60    30   -40
60    30   -30
60    30   -20
60    30   -10
60    30     0
60    30    10
60    30    20
60    30    30
60    30    40
60    30    50
60    30    60
60    30    70
60    40   -70
60    40   -60
60    40   -50
60    40   -40
60    40   -30
60    40   -20
60    40   -10
60    40     0
60    40    10
60    40    20
60    40    30
60    40    40
60    40    50
60    40    60
60    40    70
60    50   -70
60    50   -60
60    50   -50
60    50   -40
60    50   -30
60    50   -20
60    50   -10
60    50     0
60    50    10
60    50    20
60    50    30
60    50    40
60    50    50
60    50    60
60    50    70
60    60   -70
60    60   -60
60    60   -50
60    60   -40
60    60   -30
60    60   -20
60    60   -10
60    60     0
60    60    10
60    60    20
60    60    30
60    60    40
60    60    50
60    60    60
60    60    70
60    70   -70
60    70   -60
60    70   -50
60    70   -40
60    70   -30
60    70   -20
60    70   -10
60    70     0
60    70    10
60    70    20
60    70    30
60    70    40
60    70    50
60    70    60
60    70    70
60    80   -70
60    80   -60
60    80   -50
60    80   -40
60    80   -30
60    80   -20
60    80   -10
60    80     0
60    80    10
60    80    20
60    80    30
60    80    40
60    80    50
60    90   -70
60    90   -60
60    90   -50
60    90   -40
60    90   -30
60    90   -20
60    90   -10
60    90     0
60   100   -70
60   100   -60
60   100   -50
70   -70    40
70   -70    50
70   -70    60
70   -70    70
70   -60    20
70   -60    30
70   -60    40
70   -60    50
70   -60    60
70   -60    70
70   -50     0
70   -50    10
70   -50    20
70   -50    30
70   -50    40
70   -50    50
70   -50    60
70   -50    70
70   -40   -20
70   -40   -10
70   -40     0
70   -40    10
70   -40    20
70   -40    30
70   -40    40
70   -40    50
70   -40    60
70   -40    70
70   -30   -30
70   -30   -20
70   -30   -10
70   -30     0
70   -30    10
70   -30    20
70   -30    30
70   -30    40
70   -30    50
70   -30    60
70   -30    70
70   -20   -40
70   -20   -30
70   -20   -20
70   -20   -10
70   -20     0
70   -20    10
70   -20    20
70   -20    30
70   -20    40
70   -20    50
70   -20    60
70   -20    70
70   -20    80
70   -10   -50
70   -10   -40
70   -10   -30
70   -10   -20
70   -10   -10
70   -10     0
70   -10    10
70   -10    20
70   -10    30
70   -10    40
70   -10    50
70   -10    60
70   -10    70
70   -10    80
70     0   -50
70     0   -40
70     0   -30
70     0   -20
70     0   -10
70     0     0
70     0    10
70     0    20
70     0    30
70     0    40
70     0    50
70     0    60
70     0    70
70     0    80
70    10   -50
70    10   -40
70    10   -30
70    10   -20
70    10   -10
70    10     0
70    10    10
70    10    20
70    10    30
70    10    40
70    10    50
70    10    60
70    10    70
70    10    80
70    20   -50
70    20   -40
70    20   -30
70    20   -20
70    20   -10
70    20     0
70    20    10
70    20    20
70    20    30
70    20    40
70    20    50
70    20    60
70    20    70
70    20    80
70    30   -50
70    30   -40
70    30   -30
70    30   -20
70    30   -10
70    30     0
70    30    10
70    30    20
70    30    30
70    30    40
70    30    50
70    30    60
70    30    70
70    30    80
70    40   -50
70    40   -40
70    40   -30
70    40   -20
70    40   -10
70    40     0
70    40    10
70    40    20
70    40    30
70    40    40
70    40    50
70    40    60
70    40    70
70    50   -50
70    50   -40
70    50   -30
70    50   -20
70    50   -10
70    50     0
70    50    10
70    50    20
70    50    30
70    50    40
70    50    50
70    50    60
70    50    70
70    60   -50
70    60   -40
70    60   -30
70    60   -20
70    60   -10
70    60     0
70    60    10
70    60    20
70    70   -50
70    70   -40
70    70   -30
70    70   -20
70    80   -50
70    80   -40
80   -80    50
80   -80    60
80   -80    70
80   -80    80
80   -70    30
80   -70    40
80   -70    50
80   -70    60
80   -70    70
80   -70    80
80   -60    10
80   -60    20
80   -60    30
80   -60    40
80   -60    50
80   -60    60
80   -60    70
80   -60    80
80   -50   -10
80   -50     0
80   -50    10
80   -50    20
80   -50    30
80   -50    40
80   -50    50
80   -50    60
80   -50    70
80   -50    80
80   -40   -30
80   -40   -20
80   -40   -10
80   -40     0
80   -40    10
80   -40    20
80   -40    30
80   -40    40
80   -40    50
80   -40    60
80   -40    70
80   -40    80
80   -30   -40
80   -30   -30
80   -30   -20
80   -30   -10
80   -30     0
80   -30    10
80   -30    20
80   -30    30
80   -30    40
80   -30    50
80   -30    60
80   -30    70
80   -30    80
80   -20   -40
80   -20   -30
80   -20   -20
80   -20   -10
80   -20     0
80   -20    10
80   -20    20
80   -20    30
80   -20    40
80   -20    50
80   -20    60
80   -20    70
80   -20    80
80   -10   -40
80   -10   -30
80   -10   -20
80   -10   -10
80   -10     0
80   -10    10
80   -10    20
80   -10    30
80   -10    40
80   -10    50
80   -10    60
80   -10    70
80   -10    80
80     0   -40
80     0   -30
80     0   -20
80     0   -10
80     0     0
80     0    10
80     0    20
80     0    30
80     0    40
80     0    50
80     0    60
80     0    70
80     0    80
80     0    90
80    10   -40
80    10   -30
80    10   -20
80    10   -10
80    10     0
80    10    10
80    10    20
80    10    30
80    10    40
80    10    50
80    10    60
80    10    70
80    10    80
80    20   -40
80    20   -30
80    20   -20
80    20   -10
80    20     0
80    20    10
80    20    20
80    20    30
80    20    40
80    20    50
80    20    60
80    20    70
80    20    80
80    30   -40
80    30   -30
80    30   -20
80    30   -10
80    30     0
80    30    10
80    30    20
80    30    30
80    30    40
80    40   -40
80    40   -30
80    40   -20
80    40   -10
80    40     0
80    40    10
80    50   -40
80    50   -30
80    50   -20
80    50   -10
80    60   -40
90   -90    80
90   -80    40
90   -80    50
90   -80    60
90   -80    70
90   -80    80
90   -70    20
90   -70    30
90   -70    40
90   -70    50
90   -70    60
90   -70    70
90   -70    80
90   -70    90
90   -60     0
90   -60    10
90   -60    20
90   -60    30
90   -60    40
90   -60    50
90   -60    60
90   -60    70
90   -60    80
90   -60    90
90   -50   -20
90   -50   -10
90   -50     0
90   -50    10
90   -50    20
90   -50    30
90   -50    40
90   -50    50
90   -50    60
90   -50    70
90   -50    80
90   -50    90
90   -40   -20
90   -40   -10
90   -40     0
90   -40    10
90   -40    20
90   -40    30
90   -40    40
90   -40    50
90   -40    60
90   -40    70
90   -40    80
90   -40    90
90   -30   -20
90   -30   -10
90   -30     0
90   -30    10
90   -30    20
90   -30    30
90   -30    40
90   -30    50
90   -30    60
90   -30    70
90   -30    80
90   -30    90
90   -20   -20
90   -20   -10
90   -20     0
90   -20    10
90   -20    20
90   -20    30
90   -20    40
90   -20    50
90   -20    60
90   -20    70
90   -20    80
90   -20    90
90   -10   -20
90   -10   -10
90   -10     0
90   -10    10
90   -10    20
90   -10    30
90   -10    40
90   -10    50
90   -10    60
90   -10    70
90   -10    80
90   -10    90
90     0   -20
90     0   -10
90     0     0
90     0    10
90     0    20
90     0    30
90     0    40
90     0    50
90     0    60
90     0    70
90     0    80
90     0    90
90    10   -20
90    10   -10
90    10     0
90    10    10
90    10    20
90    10    30
90    10    40
90    10    50
90    20   -20
90    20   -10
90    20     0
90    20    10
90    20    20
90    30   -20
90    30   -10
100   -30     0
100   -30    10
100   -30    20
100   -30    30
100   -30    40
100   -30    50
100   -30    60
100   -30    70
100   -30    80
100   -30    90
100   -20   -10
100   -20     0
100   -20    10
100   -20    20
100   -20    30
100   -20    40
100   -20    50
100   -20    60
100   -20    70
100   -20    80
100   -20    90
100   -10   -10
100   -10     0
100   -10    10
100   -10    20
100   -10    30
100   -10    40
100   -10    50
100   -10    60
100   -10    70
100     0   -10
100     0     0
100     0    10
100     0    20
100     0    30
100    10   -10
100    10     0"""

In [ ]:
centers = matrix_str_to_np(centers_str)

In [ ]:
def compute_second_moment(L):
  Gy, Gx = np.gradient(L)
  GGTxx = Gx ** 2
  GGTxy = Gx * Gy
  GGTyy = Gy ** 2

  [r, c] = L.shape

  window_sizes = np.arange(8)**2 + 1
  s = np.sqrt(window_sizes - 1) / 2
  max_count = len(window_sizes)

  L1 = np.zeros((r, c, max_count))
  L2 = np.zeros((r, c, max_count))
  phi = np.zeros((r, c, max_count))
  pol = np.zeros((r, c, max_count))
  con = np.zeros((r, c, max_count))

  for counter, n in tqdm(enumerate(window_sizes)):
    Txx = gaussN(GGTxx, n)
    Txy = gaussN(GGTxy, n)
    Tyy = gaussN(GGTyy, n)


    tr = Txx + Tyy
    V1 = 0.5*np.emath.sqrt(tr ** 2 - 4*(Txx * Tyy - Txy ** 2))
    V1 = np.real(V1)

    L1[:, :, counter] = 0.5*tr + V1
    L2[:, :, counter] = 0.5*tr - V1

    phi[:, :, counter] = 0.5*np.arctan2(2*Txy, Txx - Tyy)

    P, angle_vector = polarity(Gx, Gy, n)

    quant_bound = angle_vector[1]/2
    for m in range(len(angle_vector)):
      a = angle_vector[-1-m]
      old_pol = pol[:, :, counter]
      Pmask = np.abs(np.cos(phi[:, :, counter] - a)) >= np.cos(quant_bound)
      Pmask = Pmask & (old_pol == 0)
      pol[:, :, counter] = old_pol + (Pmask * P[:, :, m])

    con[:, :, counter] = tr

  aniso = 1 - L2 / (L1 + eps)
  con = con / 0.25
  con = np.minimum(con, 1)
  con = np.sqrt(con)

  return L1, L2, phi, aniso, pol, con, window_sizes

In [ ]:
def conv2(x, y, mode='same'):
    return np.rot90(scipy.signal.oaconvolve(np.rot90(x, 2), np.rot90(y, 2), mode = mode), 2)

In [ ]:
def gaussN(G0, N):

  g = np.diag(np.fliplr(scipy.linalg.pascal(N)))
  g = g / np.sum(g)

  m = np.min(G0.shape)

  if m != 1:
    out = conv2(conv2(G0, g.reshape(-1, 1), 'same'), g.reshape(1, -1), 'same')
  else:
    out = scipy.signal.oaconvolve(G0, g, 'valid')

  if out.shape != G0.shape:
    x, y = out.shape
    x1, y1 = G0.shape
    out = np.pad(out, [(0, x1 - x), (0, y1 - y)])
  return out

In [ ]:
def gaussv(X, SS):
  SS = SS.astype(int)
  max_scale = np.max(SS)
  Y0 = X
  Y = Y0

  for i in range(2, max_scale + 1):
    if (-1)**i == 1:
      stencil = 0.5 * np.array([1, 1])
    else:
      stencil = 0.5 * np.array([0, 1, 1])

    Y0 = conv2(conv2(Y0, stencil.reshape(-1, 1)), stencil.reshape(1, -1))
    
    SSn = (SS == i)
    Y = (1-SSn) * Y + SSn * Y0

  return Y

In [ ]:
def polarity(Gx, Gy, N):
  angle_spacing = np.pi / 8
  num_angles = np.pi / angle_spacing
  angle_vector = np.arange(num_angles) * angle_spacing
  P = np.zeros((*tuple(Gx.shape), int(num_angles)))
  G = Gx + 1j*Gy
  for i in range(int(num_angles)):
    angle = angle_vector[i]
    Eall = np.real(G * np.exp(1j*angle))
    Eplus = Eall * (Eall >= 0)
    Eminus = -Eall * (Eall < 0)
    EplusW = gaussN(Eplus, N) + eps
    EminusW = gaussN(Eminus, N) + eps
    P[:, :, i] = np.abs(EplusW - EminusW) / (EplusW + EminusW)
  
  return P, angle_vector

In [ ]:
def texture_descriptors(L):
  L1, L2, phi, aniso, pol, con, window_sizes = compute_second_moment(L / 100)
  mask = np.mean(con, 2) > 0.05
  s = np.sqrt(window_sizes - 1) / 2
  s2 = np.concatenate((s, [4]))

  pol2 = np.copy(pol)
  for n in range(len(s)):
    N = int(4*s2[n+1]**2+1)
    pol2[:, :, n] = gaussN(pol[:, :, n], N)

  dif = np.diff(pol2, n=1, axis=2)
  thresh = 0.02
  mxm_t = np.abs(dif) <= thresh

  mxm_t[:, :, -1] = 1
  for n in range(len(s) - 1):
    mxm_t[:, :, n] = (n+1)*mxm_t[:, :, n]
  mxm_t = mxm_t + 10*np.max(s2)*(mxm_t == 0)

  min_pos = np.argmin(mxm_t, 2)
  ss = s[min_pos]
  ss = mask * ss
  SS = 4*ss**2+1
  
  aniso_s = np.zeros_like(L)
  con_s = np.zeros_like(L)
  pol_s = np.zeros_like(L)
  phi_s = np.zeros_like(L)

  for n in range(1, len(window_sizes)):
    BW = SS == window_sizes[n]
    aniso_s = (1-BW) * aniso_s + BW * aniso[:, :, n]
    con_s = (1-BW)*con_s + BW * con[:, :, n]
    pol_s = (1-BW)*pol_s+BW*pol[:, :, n]
    phi_s = (1-BW)*phi_s+BW*phi[:, :, n]

  return L1, L2, phi, phi_s, aniso, aniso_s, pol, pol_s, con, con_s, window_sizes, SS

In [ ]:
def trimedge(X, n):
  r, c = X.shape
  Y = np.zeros((r - 2*n, c-2*n))
  Y = X[n:r-n, n:c-n]
  return Y

In [ ]:
def preprocess(filename):
  xy_wt = 1
  rgb = read_image(filename)
  labL, labA, labB = rgb2lab(rgb)

  Ydim, Xdim = labL.shape

  L1, L2, phi, phi_s, aniso, aniso_s, pol, pol_s, con, con_s, window_sizes, SS = texture_descriptors(labL)

  X1 = gaussv(labL/100, SS)
  X2 = gaussv((labA + 50) / 100, SS)
  X3 = gaussv((labB + 50) / 100, SS)
  X4 = con_s * aniso_s
  X5 = con_s * pol_s
  X6 = con_s

  X7, X8 = np.meshgrid(range(0, Xdim), range(0, Ydim))
  X7 = xy_wt * X7 / np.maximum(Xdim, Ydim)
  X8 = xy_wt * X8 / np.maximum(Xdim, Ydim)

  X1 = X1 + 0.005*np.random.randn(*X1.shape)
  X2 = X2 + 0.005*np.random.randn(*X2.shape)
  X3 = X3 + 0.005*np.random.randn(*X3.shape)
  X4 = X4 + 0.005*np.random.randn(*X4.shape)
  X5 = X5 + 0.005*np.random.randn(*X5.shape)
  X6 = X6 + 0.005*np.random.randn(*X6.shape)

  X1 = trimedge(X1, 8)
  X2 = trimedge(X2, 8)
  X3 = trimedge(X3, 8)
  X4 = trimedge(X4, 8)
  X5 = trimedge(X5, 8)
  X6 = trimedge(X6, 8)
  X7 = trimedge(X7, 8)
  X8 = trimedge(X8, 8)

  A = np.stack((X1, X2, X3, X4, X5, X6, X7, X8), axis = 2)

  return A, Xdim, con_s, aniso_s, pol_s, phi_s


In [ ]:
A, Xdim, con_s, aniso_s, pol_s, phi_s = preprocess("road.jpeg")

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
gm = GaussianMixture(n_components = 10, verbose = 2, verbose_interval = 1)

In [ ]:
A_flat = A.reshape(A.shape[0]*A.shape[1], -1)

In [ ]:
pred = gm.fit_predict(A_flat)

In [ ]:
np.max(pred)

In [ ]:
from PIL import Image
im = Image.fromarray(pred / np.max(pred))
im = im.convert('RGB')
im.save("road-blobworld.jpeg")

In [ ]:
pred.reshape(A.shape[0:2]).shape

In [ ]:
plt.axis("off")
plt.imshow(pred.reshape(A.shape[0:2]), cmap="gray")
plt.savefig("road-blobworld.png", bbox_inches = 'tight')

In [ ]:
rgb = read_image("road.jpeg")
labL, labA, labB = rgb2lab(rgb)

In [ ]:
gm2 = GaussianMixture(n_components = 2, verbose = 2, verbose_interval = 1)

In [ ]:
np.stack((labL, labA, labB)).shape

In [ ]:
Aflat2 = np.stack((labL, labA, labB)).reshape(3, -1).T

In [ ]:
Aflat2.shape

In [ ]:
labL.shape

In [ ]:
pred2 = gm.fit_predict(Aflat2)

In [ ]:
plt.axis("off")
plt.imshow(pred2.reshape(labL.shape[0], labL.shape[1]), cmap="gray")
plt.savefig("road-lab.png", bbox_inches = 'tight')

In [ ]:
Ydim, Xdim = labL.shape
X7, X8 = np.meshgrid(range(0, Xdim), range(0, Ydim))
X7 = X7 / np.maximum(Xdim, Ydim)
X8 = X8 / np.maximum(Xdim, Ydim)

In [ ]:
Aflat3 = np.stack((labL, labA, labB, X7, X8)).reshape(5, -1).T

In [ ]:
pred3 = gm.fit_predict(Aflat3)

In [ ]:
plt.axis("off")
plt.imshow(pred3.reshape(labL.shape[0], labL.shape[1]), cmap="gray")
plt.savefig("road-lab-pos.png", bbox_inches = 'tight')

In [ ]:
import torch
from positional_encodings.torch_encodings import PositionalEncoding2D, Summer

In [ ]:
rgb = read_image("road.jpeg")
labL, labA, labB = rgb2lab(rgb)

In [ ]:
p_enc_2d = PositionalEncoding2D(labL.shape[0])

In [ ]:
pos_enc = p_enc_2d(torch.zeros(1, 1, *labL.shape))[0][0].numpy()

In [ ]:
Aflat4 = np.stack((labL, labA, labB, pos_enc)).reshape(4, -1).T

In [ ]:
pred4 = gm.fit_predict(Aflat4)

In [ ]:
plt.axis("off")
plt.imshow(pred4.reshape(labL.shape[0], labL.shape[0]), cmap="gray")
plt.savefig("road-lab-pos-sine.png", bbox_inches = 'tight')

In [ ]:
import torchvision

In [ ]:
vgg16 = torchvision.models.vgg16(weights='IMAGENET1K_V1')

In [ ]:
feature_extractor = vgg16.features[0:2]

In [ ]:
rgb = read_image("road.jpeg")

In [ ]:
features = feature_extractor(T.ToTensor()(rgb).unsqueeze(0))

In [ ]:
Aflat5 = features[0].permute(1, 2, 0).detach().numpy().reshape(-1, 64)

In [ ]:
pred5 = gm.fit_predict(Aflat5)

In [ ]:
final_ll = 280.81052
changes_text = """Iteration 2	 time lapse 7.70286s	 ll change 3.81175
  Iteration 3	 time lapse 7.97426s	 ll change 0.96588
  Iteration 4	 time lapse 8.05942s	 ll change 0.71437
  Iteration 5	 time lapse 8.29143s	 ll change 0.47861
  Iteration 6	 time lapse 7.79640s	 ll change 0.40831
  Iteration 7	 time lapse 7.61543s	 ll change 0.35734
  Iteration 8	 time lapse 7.23303s	 ll change 0.39797
  Iteration 9	 time lapse 7.32995s	 ll change 0.44007
  Iteration 10	 time lapse 7.32630s	 ll change 0.45355
  Iteration 11	 time lapse 7.16179s	 ll change 0.37076
  Iteration 12	 time lapse 7.24897s	 ll change 0.29243
  Iteration 13	 time lapse 7.19183s	 ll change 0.19324
  Iteration 14	 time lapse 7.06351s	 ll change 0.13303
  Iteration 15	 time lapse 7.12070s	 ll change 0.13377
  Iteration 16	 time lapse 7.31793s	 ll change 0.17741
  Iteration 17	 time lapse 7.35084s	 ll change 0.15615
  Iteration 18	 time lapse 7.14630s	 ll change 0.13796
  Iteration 19	 time lapse 7.30025s	 ll change 0.12165
  Iteration 20	 time lapse 7.23766s	 ll change 0.11303
  Iteration 21	 time lapse 7.25887s	 ll change 0.09254
  Iteration 22	 time lapse 7.24309s	 ll change 0.06381
  Iteration 23	 time lapse 7.02111s	 ll change 0.05841
  Iteration 24	 time lapse 7.12526s	 ll change 0.08126
  Iteration 25	 time lapse 7.22078s	 ll change 0.09664
  Iteration 26	 time lapse 7.11264s	 ll change 0.11544
  Iteration 27	 time lapse 7.57028s	 ll change 0.16008
  Iteration 28	 time lapse 7.43629s	 ll change 0.13590
  Iteration 29	 time lapse 7.26440s	 ll change 0.07069
  Iteration 30	 time lapse 7.51494s	 ll change 0.06268
  Iteration 31	 time lapse 7.31256s	 ll change 0.06047
  Iteration 32	 time lapse 6.84786s	 ll change 0.06201
  Iteration 33	 time lapse 6.86142s	 ll change 0.05467
  Iteration 34	 time lapse 6.84894s	 ll change 0.04374
  Iteration 35	 time lapse 6.96634s	 ll change 0.03118
  Iteration 36	 time lapse 7.31196s	 ll change 0.02844
  Iteration 37	 time lapse 7.97783s	 ll change 0.02519
  Iteration 38	 time lapse 7.99837s	 ll change 0.01576
  Iteration 39	 time lapse 7.22452s	 ll change 0.01026
  Iteration 40	 time lapse 6.91622s	 ll change 0.00726
  Iteration 41	 time lapse 6.86401s	 ll change 0.00714
  Iteration 42	 time lapse 6.98394s	 ll change 0.00777
  Iteration 43	 time lapse 7.44420s	 ll change 0.01111
  Iteration 44	 time lapse 7.11520s	 ll change 0.01328
  Iteration 45	 time lapse 7.14102s	 ll change 0.01408
  Iteration 46	 time lapse 7.15803s	 ll change 0.01380
  Iteration 47	 time lapse 7.18537s	 ll change 0.00973
  Iteration 48	 time lapse 7.42554s	 ll change 0.00539
  Iteration 49	 time lapse 7.57733s	 ll change 0.00228
  Iteration 50	 time lapse 7.54869s	 ll change 0.00110
  Iteration 51	 time lapse 6.95391s	 ll change 0.00069"""

In [ ]:
ll_changes = list(map(lambda s: float(s.split(" ")[-1]), changes_text.split("\n")))

In [ ]:
lls = [final_ll]
for change in ll_changes:
  lls.append(lls[-1] + change)
  
lls.append(400)
  
lls.reverse()

In [ ]:
lls

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.xlabel("Iteration")
plt.ylabel("Negative Log-Likelihood")
plt.plot(lls)

In [ ]:
lls

In [ ]:
plt.axis("off")
plt.imshow(pred5.reshape(labL.shape[0], labL.shape[0]), cmap="gray")
plt.savefig("road-vgg16.png", bbox_inches = 'tight')